# 0 - Load Characteristics from APM

This notebook is used to load the characteristics from APM instead from ERP.
This assumes that all characteristics that are used for the migration are already exist in ERP and no new characteristics are needed.

This step needs to be done before we execute the notebook `Indicators-03-Load-1-Sync.ipynb`.

## 1. Configuration

In [ ]:
# Pre-requisite Configuration
# ------------------------------------------------------------------------------ #
# For testing
# ------------------------------------------------------------------------------ #

%load_ext autoreload
%autoreload 2

import os

# get the current working directory
current_dir = os.getcwd()
print(f"current work directory: {current_dir}")

# if the current working directory not ends with "notebooks", change it to the parent directory
if not current_dir.endswith("notebooks"):

    workspace_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    os.chdir(workspace_root)

    print(f"changed root work directory to: {workspace_root}")

from modules.util.config import get_config_by_id, get_config_global  # noqa: E402
from modules.util.database import SQLAlchemyClient   # noqa: E402
from modules.util.helpers import Logger  # noqa: E402


# ------------------------------------------------------------------------------ #
# Configuration
# ------------------------------------------------------------------------------ #

CONFIG_ID = 'CUSTOM_TEST'
TRANSFORM = get_config_by_id(CONFIG_ID)["load"]["indicator"]
EXTRACTION_DIR = TRANSFORM["directory"]
REPORTS_DIR = f"{EXTRACTION_DIR}/reports"
CONFIG_GLOBAL = get_config_global().get('indicators').get('transform')

# ------------------------------------------------------------------------------ #
# Create directories
# ------------------------------------------------------------------------------ #
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)

# ------------------------------------------------------------------------------ #
# Logger
# ------------------------------------------------------------------------------ #

log = Logger.get_logger(CONFIG_ID)

Logger.blank_line(log)
log.info("** LOAD - INDICATORS **")
Logger.blank_line(log)

log.info(f"Extraction Directory: {EXTRACTION_DIR}")
log.info(f"Reports Directory: {REPORTS_DIR}")

db = SQLAlchemyClient(CONFIG_ID)

## 2. Load Characteristics from APM

In [ ]:
from modules.apm.manage_indicators import ApiCharacteristics
from modules.util.database import (
    ERPCharacteristics,
)

# Daten aus `data` einfügen
def insert_characteristics(data):
    characteristics = []

    try:
        # iterate over the list of characteristics
        # and create an instance of ERPCharacteristics for each one
        # and add it to the list
        for item in data.get(
            "value", []
        ):  # Extract the list of characteristics from the 'value' key
            characteristic = ERPCharacteristics(
                CharcInternalID=item["characteristicsInternalId"],
                Characteristic=item["characteristicsName"],
                CharcDataType=item["dataType"],
                CharcLength=item["charcLength"],
                CharcDecimals=item["charcDecimals"],
                UnitOfMeasureISOCode=item["unitOfMeasure"],
            )
            # add the characteristic to an array
            characteristics.append(characteristic)
        # Insert all characteristics at once
        db.insert_batches(characteristics)
        print("Successfully inserted characteristics into the database.")
    except Exception as e:
        print(f"Error while inserting data: {e}")


api_char = ApiCharacteristics(CONFIG_ID)
# get all characteristics
data = api_char.get_characteristics()

# Call the function to insert characteristics into the database
insert_characteristics(data)